In [2]:
import os
from utils import *
from os.path import join
from pprint import pprint

import numpy as np
from scipy.stats import circmean
from math import radians

import requests
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import os
import json
import model_helpers as mh
from neuron import h, load_mechanisms

--No graphics will be displayed.


In [4]:
code_version = 'Hay'
model_version = 'NeuroML'
nmldb_id = 'NMLCL000073'  # 'NMLCL000073' (Hay et al. 2011)
model_name = f'{nmldb_id}-{model_version}'
mh.download_from_nmldb(nmldb_id, model_version)

Model NMLCL000073 already downloaded.


In [5]:
sim_name = 'morphology'
cwd = os.getcwd()
models_dir = os.path.join(cwd, 'models') 
model_dir = os.path.join(models_dir, model_version, model_name)  # 'L5bPCmodelsEH')
hocs_dir = model_dir if 'biophys' not in model_name else os.path.join(model_dir,'models')
mod_dir = model_dir if 'biophys' not in model_name else os.path.join(model_dir, 'mod')
output_dir, sim_dir = mh.create_output_dirs(sim_name, model_dir)
output_dir, sim_dir

('/home/kedoxey/CRCNS/PyramidalCellSimulations/models/NeuroML/NMLCL000073-NeuroML/output',
 '/home/kedoxey/CRCNS/PyramidalCellSimulations/models/NeuroML/NMLCL000073-NeuroML/output/morphology')

In [6]:
cell_name = 'L5PC'
nml_file = os.path.join(model_dir,cell_name+'.cell.nml')
model_tree = ET.parse(nml_file)

In [7]:
spacing = 10

# NOTE: can set bounds to np.inf if desired
# NOTE: Works for cylindrical bounds

# bound in Y
if spacing == 10:
    y_lb, y_ub = -155, 155
elif spacing == 20:
    y_lb, y_ub = -310,310
else:
    pass
    

# bound in XZ-plane
rad_ub = 100

In [8]:
root = model_tree.getroot()

In [9]:
root.tag, root.attrib

('{http://www.neuroml.org/schema/neuroml2}neuroml',
 {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.neuroml.org/schema/neuroml2  https://raw.githubusercontent.com/NeuroML/NeuroML2/development/Schemas/NeuroML2/NeuroML_v2beta4.xsd',
  'id': 'L5PC'})

In [10]:
for child in root:
    print(child.tag, child.attrib)

{http://www.neuroml.org/schema/neuroml2}include {'href': 'Ca_HVA.channel.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'Ca_LVAst.channel.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'CaDynamics_E2_NML2__decay122__gamma5_09Emin4.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'CaDynamics_E2_NML2__decay460__gamma5_01Emin4.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'Ih.channel.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'Im.channel.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'K_Pst.channel.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'K_Tst.channel.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'Nap_Et2.channel.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'NaTa_t.channel.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'pas.channel.nml'}
{http://www.neuroml.org/schema/neuroml2}include {'href': 'SK_E2.channel.nml'}
{ht

In [11]:
cell= root[-1]
cell

<Element '{http://www.neuroml.org/schema/neuroml2}cell' at 0x7f7e0131e950>

In [12]:
for child in cell:
    print(child.tag, child.attrib)

{http://www.neuroml.org/schema/neuroml2}notes {}
{http://www.neuroml.org/schema/neuroml2}morphology {'id': 'morphology_L5PC'}
{http://www.neuroml.org/schema/neuroml2}biophysicalProperties {'id': 'biophys'}


## Morphology

In [13]:
morpho = cell[1]
morpho

<Element '{http://www.neuroml.org/schema/neuroml2}morphology' at 0x7f7e0131e8b0>

In [14]:
segmentGroups = {}
for i, group in enumerate(morpho.findall('{http://www.neuroml.org/schema/neuroml2}segmentGroup')):
    
    segmentGroups.update({i:group})


In [15]:
"k8".isdigit()

False

### Segment names

In [16]:
seg_names = []
for i in segmentGroups:
    segmentGroup = segmentGroups[i]
    id = segmentGroup.attrib['id']

    if any(map(str.isdigit, id)):
        name = id.rsplit('_', 1)[0]
    else:
        name = id
    # name = id
    if name not in seg_names:
        seg_names.append(name)
seg_names

['soma',
 'axon',
 'apic',
 'dend',
 'all',
 'soma_group',
 'apical_dends_and_soma',
 'dends_and_soma',
 'soma_and_axon',
 'basal_dends_and_soma',
 'axon_group',
 'dendrite_group',
 'apical_dends',
 'non_hot_apical_dends',
 'OneSecGrp_SectionRef',
 'basal_dends']

In [17]:
comp_domain_names = ['soma_group', 'axon_group', 'apical_dends', 'basal_dends']
comp_domains = {}
for key, val in segmentGroups.items():
    if val.attrib['id'] in comp_domain_names:
        comp_domains.update({val.attrib['id']:val})

In [18]:
comp_domains

{'soma_group': <Element '{http://www.neuroml.org/schema/neuroml2}segmentGroup' at 0x7f7dffc6fdb0>,
 'axon_group': <Element '{http://www.neuroml.org/schema/neuroml2}segmentGroup' at 0x7f7dffc2cb80>,
 'apical_dends': <Element '{http://www.neuroml.org/schema/neuroml2}segmentGroup' at 0x7f7dffc4a900>,
 'basal_dends': <Element '{http://www.neuroml.org/schema/neuroml2}segmentGroup' at 0x7f7dffbeb720>}

## Somatic compartments

In [19]:
comp_domains['soma_group']

<Element '{http://www.neuroml.org/schema/neuroml2}segmentGroup' at 0x7f7dffc6fdb0>

In [20]:
for child in comp_domains['soma_group']:
    print(child.tag, child.attrib)

{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'soma_0'}


In [21]:
soma_group = comp_domains['soma_group'][0].attrib['segmentGroup']

In [22]:
soma_segments = {}
for segment in morpho.findall('{http://www.neuroml.org/schema/neuroml2}segment'):
    if soma_group in segment.attrib['name']:
        soma_segments.update({segment.attrib['id']:segment.attrib['name']})
soma_segments

{'0': 'Seg0_soma_0',
 '2': 'Seg1_soma_0',
 '3': 'Seg2_soma_0',
 '4': 'Seg3_soma_0',
 '5': 'Seg4_soma_0',
 '6': 'Seg5_soma_0',
 '7': 'Seg6_soma_0',
 '8': 'Seg7_soma_0',
 '9': 'Seg8_soma_0',
 '10': 'Seg9_soma_0',
 '11': 'Seg10_soma_0',
 '12': 'Seg11_soma_0',
 '13': 'Seg12_soma_0',
 '14': 'Seg13_soma_0',
 '15': 'Seg14_soma_0',
 '16': 'Seg15_soma_0',
 '17': 'Seg16_soma_0',
 '18': 'Seg17_soma_0',
 '19': 'Seg18_soma_0',
 '20': 'Seg19_soma_0'}

In [23]:
for child in morpho[0]:
    print(child.tag,child.attrib)

{http://www.neuroml.org/schema/neuroml2}proximal {'x': '34.1634', 'y': '17.6215', 'z': '-50.25', 'diameter': '3.80039'}
{http://www.neuroml.org/schema/neuroml2}distal {'x': '35.3196', 'y': '17.6937', 'z': '-50.25', 'diameter': '6.44373'}


### Soma length and y-bounds

In [24]:
these_segments = soma_segments
soma_length = 0
soma_parents = {}

for segment in morpho.findall('{http://www.neuroml.org/schema/neuroml2}segment'):
    seg_id = segment.attrib['id']
    
    if seg_id in these_segments.keys():
        
        if 'Seg0' in segment.attrib['name']:
            proximal = segment[0]
            distal = segment[1]

            soma_y_prox = float(proximal.attrib['y'])
            soma_y_dist = float(distal.attrib['y'])

            if soma_y_prox < soma_y_dist:
                lb = soma_y_prox
                ub = soma_y_dist
                soma_dir = 1  # prox->dist
                soma_root = lb
            else:
                lb = soma_y_dist
                ub = soma_y_prox
                soma_dir = -1  # dist->prox
                soma_root = ub

        else:
            parent_id = segment[0].attrib['segment']

            proximal = soma_parents[parent_id]
            distal = segment[1]

            if soma_dir==1 and float(distal.attrib['y'])>ub:
                ub = float(distal.attrib['y'])
            if soma_dir==1 and float(distal.attrib['y'])<lb:
                lb = float(distal.attrib['y'])  
        
        soma_parents.update({seg_id: distal})

        soma_length += compute_segment_length(proximal, distal)

soma_center = lb + (ub-lb)/2
# soma_length = ub-lb

In [25]:
soma_center, lb, ub, soma_length, ub-lb

(18.34365, 17.6215, 19.0658, 23.169360230727168, 1.4442999999999984)

### Soma width and bounds

In [26]:
these_segments = soma_segments
soma_width = 0
soma_parents = {}

for segment in morpho.findall('{http://www.neuroml.org/schema/neuroml2}segment'):
    seg_id = segment.attrib['id']
    
    if seg_id in these_segments.keys():
        
        if 'Seg0' in segment.attrib['name']:
            proximal = segment[0]
            distal = segment[1]

            soma_x_prox = float(proximal.attrib['x'])
            soma_x_dist = float(distal.attrib['x'])

            if soma_x_prox < soma_x_dist:
                lb = soma_x_prox
                ub = soma_x_dist
                soma_dir = 1  # prox->dist
                soma_root = lb
            else:
                lb = soma_x_dist
                ub = soma_x_prox
                soma_dir = -1  # dist->prox
                soma_root = ub

        else:
            parent_id = segment[0].attrib['segment']

            proximal = soma_parents[parent_id]
            distal = segment[1]

            if soma_dir==1 and float(distal.attrib['x'])>ub:
                ub = float(distal.attrib['x'])
            if soma_dir==1 and float(distal.attrib['x'])<lb:
                lb = float(distal.attrib['x'])  
        
        soma_parents.update({seg_id: distal})

        soma_width += compute_segment_length(proximal, distal)

soma_center = lb + (ub-lb)/2

In [27]:
soma_center, lb, ub, soma_width, ub-lb

(45.72555, 34.1634, 57.2877, 23.169360230727168, 23.124299999999998)

## Stems

In [28]:
stems = {}

for segment in morpho.findall('{http://www.neuroml.org/schema/neuroml2}segment'):
    seg_id = segment.attrib['id']
    seg_name = segment.attrib['name']

    if not seg_name in soma_segments.values():

        parent_id = segment[0].attrib['segment']

        if parent_id in soma_parents.keys():
            stems.update({seg_id: seg_name})
    
print(f'Number of stems = {len(stems)}')
stems

Number of stems = 10


{'4067': 'Seg0_axon_0',
 '1660': 'Seg0_apic_0',
 '1594': 'Seg0_dend_79',
 '1585': 'Seg0_dend_78',
 '1260': 'Seg0_dend_63',
 '827': 'Seg0_dend_42',
 '749': 'Seg0_dend_39',
 '353': 'Seg0_dend_16',
 '206': 'Seg0_dend_7',
 '21': 'Seg0_dend_0'}

### compute stem cross-sectional areas

In [29]:
stem_names = []
stem_diams = []

for segment in morpho.findall('{http://www.neuroml.org/schema/neuroml2}segment'):
    seg_name = segment.attrib['name']

    if seg_name in stems.values():
                   
        proximal = segment[1]
        diameter = float(proximal.attrib['diameter'])
        stem_names.append(seg_name)
        stem_diams.append(diameter)

stem_diams = np.array(stem_diams)
stem_diams

array([ 1.  , 10.8 ,  0.88,  0.29,  1.46,  0.58,  0.88,  1.46,  0.29,
        0.58])

In [30]:
def compute_stem_csa(diam):
    return np.pi*np.square(diam/2)

In [31]:
stem_csas = compute_stem_csa(stem_diams)
stem_csas

array([7.85398163e-01, 9.16088418e+01, 6.08212338e-01, 6.60519855e-02,
       1.67415473e+00, 2.64207942e-01, 6.08212338e-01, 1.67415473e+00,
       6.60519855e-02, 2.64207942e-01])

In [32]:
stem_names

['Seg0_axon_0',
 'Seg0_apic_0',
 'Seg0_dend_79',
 'Seg0_dend_78',
 'Seg0_dend_63',
 'Seg0_dend_42',
 'Seg0_dend_39',
 'Seg0_dend_16',
 'Seg0_dend_7',
 'Seg0_dend_0']

In [33]:
np.sum(stem_csas)

97.61949392315906

In [34]:
for stem_id,name in stems.items():
    
    prox_list, dist_list = get_segment_coord_lists({stem_id:name},morpho,stems) 
    print(name,' angle:',compute_average_orientation(prox_list,dist_list)) # mean projection angle of all stem compartments off XZ-plane? -- needs to be restricted to local compartments e.g., apic


Seg0_axon_0  angle: 0.51678330799935
Seg0_apic_0  angle: 0.5771213449789857
Seg0_dend_79  angle: 0.34116452615574666
Seg0_dend_78  angle: 0.2050942633818013
Seg0_dend_63  angle: 0.1575069491200203
Seg0_dend_42  angle: 0.3198006361688046
Seg0_dend_39  angle: 0.2712484457812747
Seg0_dend_16  angle: 0.21440570451713192
Seg0_dend_7  angle: 0.24256382200546311
Seg0_dend_0  angle: 0.2818319164172626


## Axon compartments

In [35]:
for child in comp_domains['axon_group']:
    print(child.tag,child.attrib)

{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'axon_0'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'axon_1'}


In [36]:
domain = 'axon_group'

axon_groups = []

for seg_group in comp_domains[domain]:
    axon_groups.append(seg_group.attrib['segmentGroup'])

axon_groups

['axon_0', 'axon_1']

In [37]:
axon_segments = {}

for segment in morpho.findall('{http://www.neuroml.org/schema/neuroml2}segment'):
    
    # leverage standardized segment naming
    seg_name = segment.attrib['name'].split('_')
    this_group = seg_name[1]+'_'+seg_name[2]
    
    if this_group in axon_groups:
        axon_segments.update({segment.attrib['id']:segment.attrib['name']})

axon_segments

{'4067': 'Seg0_axon_0',
 '4068': 'Seg1_axon_0',
 '4069': 'Seg0_axon_1',
 '4070': 'Seg1_axon_1'}

In [38]:
stem_groups = [s.split('_')[1]+'_'+s.split('_')[2] for s in stems.values()]
stem_groups

['axon_0',
 'apic_0',
 'dend_79',
 'dend_78',
 'dend_63',
 'dend_42',
 'dend_39',
 'dend_16',
 'dend_7',
 'dend_0']

In [39]:
these_segments = axon_segments
axon_length = 0
axon_parents = {}

# for compute domain-level props
proximal_list = []
distal_list = []

for segment in morpho.findall('{http://www.neuroml.org/schema/neuroml2}segment'):
    seg_id = segment.attrib['id']
    seg_name = segment.attrib['name']

    if seg_name in these_segments.values():
        
        print(len(segment))
        print(seg_name)
        if len(segment)==2:
            print('... is a middle segment')
        if len(segment)==3:
            print('... is stem or branch segment')
    
        # all segments have parent id
        parent_id = segment[0].attrib['segment']
        
        
        # if not stem of soma, look at parent for proximal
        if seg_name in stems.values(): # this will ignore branching -- NEED TO BE FIXED
            
            # parent is segment[0]
            for seg_element in segment:
                if 'proximal' in seg_element.tag:
                    proximal = seg_element
                if 'distal' in seg_element.tag:
                    distal = seg_element
            
#             proximal = segment[1]
#             distal = segment[2]
        else:
            # no proximal so distal is segment[1] for non-branch segments
#             proximal = axon_parents[parent_id]
#             distal = segment[1] # old
            for seg_element in segment:
                if 'proximal' in seg_element.tag:
                    proximal = seg_element
                elif 'distal' in seg_element.tag:
                    distal = seg_element
                else:
                    proximal = axon_parents[parent_id]
            
            
        proximal_list.append(proximal)
        distal_list.append(distal)
            
        axon_parents.update({seg_id:distal})
        
        # test
        axon_length += compute_segment_length(proximal,distal)
        
        

3
Seg0_axon_0
... is stem or branch segment
2
Seg1_axon_0
... is a middle segment
3
Seg0_axon_1
... is stem or branch segment
2
Seg1_axon_1
... is a middle segment


In [40]:
axon_length

59.999951281683465

In [41]:
x0, y0, z0 = float(proximal_list[0].attrib['x']), float(proximal_list[0].attrib['y']), float(proximal_list[0].attrib['z'])
xf, yf, zf = float(distal_list[-1].attrib['x']), float(distal_list[-1].attrib['y']), float(distal_list[-1].attrib['z'])

In [42]:
r = np.sqrt((xf-x0)**2 + (yf-y0)**2 + (zf-z0)**2)
r

59.99995128164689

In [43]:
yf-y0

56.6126

In [44]:
compute_average_orientation(proximal_list,distal_list) # angle from XZ-plane


0.7068098819832662

## Apical compartments

In [45]:
for i, child in enumerate(comp_domains['apical_dends']):
    print(child.tag,child.attrib)
    
    if i==10:
        print('...')
        break

{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_0'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_104'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_1'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_106'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_105'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_99'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_2'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_108'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_107'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_103'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'apic_100'}
...


In [46]:
apical_groups = get_domain_groups('apical_dends',comp_domains)

In [47]:
y_lb, y_ub, rad_ub

(-155, 155, 100)

In [48]:
apical_init_groups = [group for group in apical_groups if group in stem_groups]
apical_init_groups

['apic_0']

In [49]:
# get apical segments within cylindrical bounds
# apical_segments = get_domain_segments(morpho,apical_groups,bounds=[[y_lb,y_ub],rad_ub])

upper_apical_segments = get_domain_segments(morpho,apical_init_groups,bounds=[[0,np.inf],np.inf])
lower_apical_segments = get_domain_segments(morpho,apical_init_groups,bounds=[[-np.inf,0],np.inf])


In [50]:
upper_apical_segments

{'1660': 'Seg0_apic_0',
 '1662': 'Seg2_apic_0',
 '1663': 'Seg3_apic_0',
 '1664': 'Seg4_apic_0',
 '1665': 'Seg5_apic_0',
 '1666': 'Seg6_apic_0',
 '1667': 'Seg7_apic_0',
 '1668': 'Seg8_apic_0',
 '1669': 'Seg9_apic_0',
 '1670': 'Seg10_apic_0',
 '1671': 'Seg11_apic_0'}

In [51]:
lower_apical_segments

{}

In [52]:
apical_segments = upper_apical_segments

In [53]:
proximal_list, distal_list = get_segment_coord_lists(apical_segments,morpho,stems,verbose=True)

Removed 0 segments from list...


In [54]:
compute_average_orientation(proximal_list,distal_list) 

0.8853862397650601

In [55]:
compute_total_length(proximal_list,distal_list)

46.13561135987528

In [56]:
compute_average_distance(distal_list)

4149.221884153519

## Basal compartments

In [57]:
for i, child in enumerate(comp_domains['basal_dends']):
    print(child.tag,child.attrib)
    
    if i==10:
        print('...')
        break

{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_79'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_78'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_63'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_42'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_39'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_16'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_7'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_0'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_81'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_80'}
{http://www.neuroml.org/schema/neuroml2}include {'segmentGroup': 'dend_71'}
...


In [58]:
basal_groups = get_domain_groups('basal_dends',comp_domains)
# basal_segments = get_domain_segments(morpho,basal_groups,bounds=[[y_lb,y_ub],rad_ub])
basal_segments = get_domain_segments(morpho,basal_groups,bounds=None)
proximal_list, distal_list = get_segment_coord_lists(basal_segments,morpho,stems,verbose=True)

Removed 0 segments from list...


## Stem ends

In [61]:
# compute max extent for the different compartment domains
seg_ends = []
lower_segs = []
for seg, distal_end in zip(basal_segments.items(),distal_list):
    y = float(distal_end.attrib['y'])
    seg_ends.append(y)
    
    if y<0: lower_segs.append(seg)
    
upper_y_dist, lower_y_dist = np.max(seg_ends), np.min(seg_ends)



In [62]:
upper_y_dist, lower_y_dist

(116.357, -190.093)

In [63]:
def get_domain_locs(comp_domains,morpho,stems,which_domain='basal'):
    
    
    domain_groups = get_domain_groups(which_domain,comp_domains)
    domain_segments = get_domain_segments(morpho,domain_groups,bounds=None)
    proximal_list, distal_list = get_segment_coord_lists(domain_segments,morpho,stems,verbose=False)
    
    locs = np.zeros(len(proximal_list),3)
    diams = np.zeros(len(proximal))
    
    for i, (prox_end,distal_end) in enumerate(zip(proximal_list,distal_list)):
        xd, yd, zd = float(distal_end.attrib['x']),float(distal_end.attrib['y']),float(distal_end.attrib['z'])
        xp, yp, zp = float(prox_end.attrib['x']),float(prox_end.attrib['y']),float(prox_end.attrib['z'])

        locs[i,0], locs[i,1], locs[i,2] = (xd-xp)/2.,(yd-yp)/2.,(zd-zp)/2.

        diams[i] = float(prox_end.attrib['diameter'])
    
    
    return locs, diams

In [65]:
def get_terminal_segments(morpho,comp_domains,which_domain='basal',verbose=False):
    
    domain_groups = get_domain_groups(which_domain,comp_domains)
    domain_segments = get_domain_segments(morpho,domain_groups,bounds=None)
    
    no_children = domain_segments.copy()
    
    
    for segment in morpho.findall('{http://www.neuroml.org/schema/neuroml2}segment'):
        seg_id = segment.attrib['id']
        seg_name = segment.attrib['name']

        
        # find parents
        if seg_name in domain_segments.values():
            parent_id = segment[0].attrib['segment'] 
            
            # if it is a parent, then remove it
            try:
                no_children.pop(parent_id)
            except KeyError:
                if verbose: print('Parent is not in %s domain.'%which_domain)
            
    return no_children

In [66]:
def get_terminal_coord_lists(terminal_segments,domain_segments,morpho,stems,verbose=False):
    segment_parents = {}
    no_parent = {}

    # for compute domain-level props
    proximal_list = []
    distal_list = []

    for segment in morpho.findall('{http://www.neuroml.org/schema/neuroml2}segment'):
        seg_id = segment.attrib['id']
        seg_name = segment.attrib['name']


        if seg_name in domain_segments.values():

            parent_id = segment[0].attrib['segment']

            # if not stem of soma, look at parent for proximal
            if seg_name in stems.values():

                # parent is segment[0]
                proximal = segment[1]
                distal = segment[2]
                
                segment_parents.update({seg_id:distal})
            else:
                # no proximal so distal is segment[1]
                distal = segment[1]
                segment_parents.update({seg_id:distal})
                
                # sometimes parent segment is out of bounds so not included
                try:
                    proximal = segment_parents[parent_id]
            
                except KeyError:
                    no_parent.update({seg_id:{}})
                    no_parent[seg_id].update({'Name':seg_name})
                    no_parent[seg_id].update({'Parent ID':parent_id})
                    
                    # don't append anything to list
                    continue
            
            if seg_name in terminal_segments.values():
                proximal_list.append(proximal)
                distal_list.append(distal)
                
    return proximal_list,distal_list

In [64]:
def get_stem_terminal_metrics(comp_domains,morpho,stems,which_stem_domain='basal',soma_center=0):
    """
        Only meant for basal or non-basal/apical.
    """
    
    stem_domains = [s.split('_')[1]+'_'+s.split('_')[2] for s in stems.values()]
    
    stem_groups = get_domain_groups(which_stem_domain,comp_domains)
    stem_segments = get_domain_segments(morpho,stem_groups,bounds=None)
    terminal_segs = get_terminal_segments(morpho,comp_domains,which_domain=which_stem_domain)
    
    proximal_list, distal_list = get_terminal_coord_lists(terminal_segs,stem_segments,morpho,stems,verbose=False)
    
    
  
    lower_yfs = []
    lower_xzfs = []
    upper_yfs = []
    upper_xzfs = []
    upper_angles = []
    lower_angles = []

    for seg, distal_end, prox_end in zip(terminal_segs.items(),distal_list,proximal_list):
        xd, yd, zd = float(distal_end.attrib['x']),float(distal_end.attrib['y']),float(distal_end.attrib['z'])
        xp, yp, zp = float(prox_end.attrib['x']),float(prox_end.attrib['y']),float(prox_end.attrib['z'])
        
        
        # project all segments onto XZ-plane
        dist_xz = np.sqrt(np.square(xd)**2 + np.square(zd)**2)

        # compute signed angle in radians
        dist_y = yd
        angle = np.arctan2(dist_y,dist_xz)

        

        if yp<soma_center: 
            lower_yfs.append(yd)
            lower_angles.append(angle)
            lower_xzfs.append(dist_xz)

        else: 
            upper_yfs.append(yd)
            upper_angles.append(angle)
            upper_xzfs.append(dist_xz)



    return lower_yfs, lower_xzfs, upper_yfs, upper_xzfs, [circmean(lower_angles),circmean(upper_angles),np.mean(lower_yfs),np.mean(upper_yfs)]



In [68]:
axon_groups = get_domain_groups('axon_group',comp_domains)
axon_segments = get_domain_segments(morpho,axon_groups,bounds=None)
axon_segments

In [71]:
axon_terminal_segs = get_terminal_segments(morpho,comp_domains,which_domain='axon_group')
axon_terminal_segs

{'4070': 'Seg1_axon_1'}

In [72]:
proximal_list, distal_list = get_terminal_coord_lists(axon_terminal_segs,axon_segments,morpho,stems,verbose=False)
proximal_list

[<Element '{http://www.neuroml.org/schema/neuroml2}proximal' at 0x7f7e0087cf40>]